# <font color=green>1. MEU PRIMEIRO SCRAPING

## 1.1. Introdução

*Web Scraping* é o termo utilizado para definir a prática de coletar automaticamente informações na Internet. Isto é feito, geralmente, por meio de programas que simulam a navegação humana na Web.

## 1.2. Ambiente e bibliotecas
### Utilizaremos em nosso treinamento o navegador Google Chrome

In [1]:
import bs4
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import urlopen
import pandas
import numpy as np

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pandas.__version__)

BeautifulSoup -> 4.11.1
urllib -> 3.8
pandas -> 1.2.5


In [5]:
precos = [579, 791.2, 464.4, 585.07, 465.4, 489.9, 465.64]
preco_bemol = 519

mediana = np.median(precos)

dif = round((1 - (preco_bemol/mediana)) * 100, 1)

print(mediana, dif)

489.9 5.6


---
# <font color=green>2. OBTENDO E TRATANDO O CONTEÚDO DE UM HTML

## 2.1. Entendendo a web

<img src="./web/web.png" width="700">

## 2.2. Obtendo o conteúdo HTML de um site

### urllib.request
#### https://docs.python.org/3/library/urllib.html
#### https://docs.python.org/3/library/urllib.request.html#urllib.request.Request

In [2]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

url = 'https://www.pokemon.com/br/pokedex/'

#Definindo headers para permissões de navegador. User-Agent é encontrado dentro da opção de inspecionar página -> Network -> arquivo da página -> Request Headers
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

#Fazendo tentativa de obter o resultado da requisição, caso retorne erro de protocolo ou na url, cai na exceção e retorna o erro
try:
    req = Request(url, headers=headers)
    response = urlopen(req)
    html = response.read()
    #print(html)

except HTTPError as e:
    print(e.status, e.reason)

except URLError as e:
  print(e.reason)


## 2.3. Tratamento de string

### Convertando o tipo bytes para string

In [3]:
#Observamos que inicialmente os dados obtidos são do formato bytes. Então para facilitar a manipulação, pode-se converter para tipo string
type(html)

bytes

In [4]:
#Tratando caracteres
html = html.decode('utf-8')
type(html)

str

### Eliminando os caracteres de tabulação, quebra de linha etc.

In [5]:
#Função que identifica todos os elementos que estão separados por espaços, quebra de linha, etc
html.split()

['<!DOCTYPE',
 'html>',
 '<html',
 'class="no-js',
 '"',
 'lang="pt-br">',
 '<head>',
 '<meta',
 'charset="utf-8"',
 '/>',
 '<meta',
 'http-equiv="X-UA-Compatible"',
 'content="IE=edge,chrome=1"',
 '/><script',
 'type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={xpid:"VQ4OWFZXGwIAXFZTBgI=",licenseKey:"ba34eb72cb",applicationID:"1087113"};;(()=>{var',
 'e,t,r={9071:(e,t,r)=>{"use',
 'strict";r.d(t,{I:()=>n});var',
 'n=0,i=navigator.userAgent.match(/Firefox[\\/\\s](\\d+\\.\\d+)/);i&&(n=+i[1])},6900:(e,t,r)=>{"use',
 'strict";let',
 'n;if(r.d(t,{H:()=>i}),r(2374).il){const',
 'e=document.createElement("div");e.innerHTML="\\x3c!--[if',
 'lte',
 'IE',
 '6]><div></div><![endif]--\\x3e\\x3c!--[if',
 'lte',
 'IE',
 '7]><div></div><![endif]--\\x3e\\x3c!--[if',
 'lte',
 'IE',
 '8]><div></div><![endif]--\\x3e\\x3c!--[if',
 'lte',
 'IE',
 '9]><div></div><![endif]--\\x3e",n=e.getElementsByTagName("div").length}var

### Função de tratamento de strings

In [6]:
#Função para separar os dados porém com apenas um espaço entre eles. Após isso, remover os espaços em branco entre as tags
def treat_html(input):  
    return " ".join(input.split()).replace('> <', '><')

In [7]:
html = treat_html(html)
#html

---
# <font color=green>3. INTRODUÇÃO AO BEAUTIFULSOUP

## 3.1. HTML da nossa página

**HTML** (*HyperText Markup Language*) é uma linguagem de marcação composta por **tags** que deteminam o papel que cada parte do documento vai assumir. As **tags** são formadas pelo seu nome e atributos. Os atributos servem para configurar e também modificar as características padrões de uma **tag**.

## Estrutura básica

```html
<html>
    <head>
        <meta charset="utf-8" />
        <title>Alura Motors</title>
    </head>
    <body>
        <div id="container">
            <h1>Alura</h1>
            <h2 class="formato">Cursos de Tecnologia</h2>
            <p>Você vai estudar, praticar, discutir e aprender.</p>
            <a href="https://www.alura.com.br/">Clique aqui</a>
        </div>
    </body>
</html>
```

```<html>``` - determina o início do documento.

```<head>``` - cabeçalho. Contém informações e configurações do documento.

```<body>``` - é o corpo do documento, onde todo o conteúdo é colocado. Esta é a parte visível em um navegador.

## Tags mais comuns

```<div>``` - define uma divisão da página. Pode ser formatada de diversas maneiras.

```<h1>, <h2>, <h3>, <h4>, <h5>, <h6>``` - marcadores de títulos.

```<p>``` - marcador de parágrafo.

```<a>``` - hiperlink.

```<img>``` - exibição de imagens.

```<table>``` - definição de tabelas.

```<ul>, <li>``` - definição de listas.


## 3.2. Criando um objeto BeautifulSoup
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

### Pokedex: https://www.pokemon.com/br/pokedex/

In [8]:
soup = BeautifulSoup(html, 'html.parser')
print(type(soup))
print(soup.prettify()) #Comando que mostra o html identado

<class 'bs4.BeautifulSoup'>
<!DOCTYPE html>
<html class="no-js" lang="pt-br">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={xpid:"VQ4OWFZXGwIAXFZTBgI=",licenseKey:"ba34eb72cb",applicationID:"1087113"};;(()=>{var e,t,r={9071:(e,t,r)=>{"use strict";r.d(t,{I:()=>n});var n=0,i=navigator.userAgent.match(/Firefox[\/\s](\d+\.\d+)/);i&&(n=+i[1])},6900:(e,t,r)=>{"use strict";let n;if(r.d(t,{H:()=>i}),r(2374).il){const e=document.createElement("div");e.innerHTML="\x3c!--[if lte IE 6]><div></div><![endif]--\x3e\x3c!--[if lte IE 7]><div></div><![endif]--\x3e\x3c!--[if lte IE 8]><div></div><![endif]--\x3e\x3c!--[if lte IE 9]><div></div><![endif]--\x3e",n=e.getElementsByTagName("div").length}var i;i=4===n?6:3===n?7:2===n?8:1===n?9:0},6562:(e,t,r)=>{"use strict";r.d(t,{P_:()=>p,Mt:()=>v,C5:()=>d,DL:(

## 3.3. Acessando tags

Digitando soup + a tag que queira encontrar, ele busca a primeira ocorrência dessa tag e exibe os registros que estão dentro dela

In [9]:
soup.title

<title> Pokédex | www.pokemon.net.br</title>

In [10]:
soup.body.div.div

<div class="pokemon-gus-container" data-api="https://www.pokemon.com/api/gus/" data-locale="pt-br" data-site="pcom" data-width="913px"></div>

In [11]:
#Comando que traz somente o conteúdo da tag - get_text() ou getText()
soup.title.get_text()

' Pokédex | www.pokemon.net.br'

## 3.4. Acessando os atributos de uma tag

In [12]:
soup.body.div.div.attrs

{'class': ['pokemon-gus-container'],
 'data-site': 'pcom',
 'data-locale': 'pt-br',
 'data-api': 'https://www.pokemon.com/api/gus/',
 'data-width': '913px'}

In [13]:
soup.body.div.div.attrs.keys()

dict_keys(['class', 'data-site', 'data-locale', 'data-api', 'data-width'])

In [14]:
soup.body.div.div.attrs.values()

dict_values([['pokemon-gus-container'], 'pcom', 'pt-br', 'https://www.pokemon.com/api/gus/', '913px'])

In [15]:
soup.body.div.div.attrs['class']

['pokemon-gus-container']

In [16]:
soup.body.div.div.get('data-api')

'https://www.pokemon.com/api/gus/'

---
# <font color=green>4. PESQUISANDO COM O BEAUTIFULSOUP

## 4.1. Os métodos *find()* e *findAll()*

- ### *find(tag, attributes, recursive, text, **kwargs)*

- ### *findAll(tag, attributes, recursive, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all

> **Observação:**
> - *findAll()* também pode ser utilizado como *find_all()*

### Método *find()*

In [17]:
#Encontra a primeira ocorrência da tag buscada
soup.find('h2')

<h2>Realize a busca por Pokémon pelo nome ou usando o número do Pokédex Nacional.</h2>

### Método *findAll()*

In [18]:
#Encontra todas as ocorrências da tag buscada
soup.findAll('h2')

[<h2>Realize a busca por Pokémon pelo nome ou usando o número do Pokédex Nacional.</h2>,
 <h2 class="section-title no-padding"> Tipo e Fraqueza </h2>,
 <h2>National Pokédex</h2>,
 <h2><span class="offscreen">The Pokémon Company</span></h2>]

In [19]:
#Limitando ocorrências
soup.findAll('h2', limit=3)

[<h2>Realize a busca por Pokémon pelo nome ou usando o número do Pokédex Nacional.</h2>,
 <h2 class="section-title no-padding"> Tipo e Fraqueza </h2>,
 <h2>National Pokédex</h2>]

In [20]:
#Limitando ocorrências
soup.findAll('h2', limit=3)[2]

<h2>National Pokédex</h2>

### Atalho para o método *findAll()*

In [21]:
soup('h2')

[<h2>Realize a busca por Pokémon pelo nome ou usando o número do Pokédex Nacional.</h2>,
 <h2 class="section-title no-padding"> Tipo e Fraqueza </h2>,
 <h2>National Pokédex</h2>,
 <h2><span class="offscreen">The Pokémon Company</span></h2>]

### Passando listas de TAGs

In [22]:
soup.findAll(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

[<h1 class="section-title no-margin no-padding">Pokédex</h1>,
 <h2>Realize a busca por Pokémon pelo nome ou usando o número do Pokédex Nacional.</h2>,
 <h2 class="section-title no-padding"> Tipo e Fraqueza </h2>,
 <h3 class="rangeTitle">Intervalo de números</h3>,
 <h3 class="section-title pokedex-filter-title">Habilidade</h3>,
 <h3 class="section-title pokedex-filter-title">Altura</h3>,
 <h3 class="section-title pokedex-filter-title">Peso</h3>,
 <h3 class="color-text color-gray sort-label">Organizar por</h3>,
 <h3>Nenhum Pokémon corresponde à sua pesquisa!</h3>,
 <h2>National Pokédex</h2>,
 <h3>É necessário acessar sua conta</h3>,
 <h2><span class="offscreen">The Pokémon Company</span></h2>,
 <h6> Você está prestes a sair do site operado pela The Pokémon Company International, Inc. </h6>,
 <h6> Clique em "Continuar" para visitar PokemonCenter.com, nossa loja online oficial. </h6>,
 <h6> As políticas de privacidade e as práticas de segurança são diferentes. </h6>,
 <h6>Reportar nome de 

### Utilizando o argumento *attributes*

In [23]:
#Encontrando tags com um atributo de nome específico
soup.findAll('h3', attrs={'class':'section-title pokedex-filter-title'})

[<h3 class="section-title pokedex-filter-title">Habilidade</h3>,
 <h3 class="section-title pokedex-filter-title">Altura</h3>,
 <h3 class="section-title pokedex-filter-title">Peso</h3>]

### Buscando por conteúdo de uma TAG

In [24]:
soup.findAll('h1', text='Pokédex')

[<h1 class="section-title no-margin no-padding">Pokédex</h1>]

In [25]:
import requests 
url = 'https://www.pokemon.com/br/pokedex/bulbasaur'

# Fazendo a requisição HTTP para a URL
r = requests.get(url)

#Extraindo o conteúdo HTML da página
soup = BeautifulSoup(r.content, 'html.parser')

soup.findAll('ul', attrs={'class':'evolution-profile'})

[<ul class="evolution-profile match-height-tablet">
 <li class="first">
 <a href="/br/pokedex/bulbasaur">
 <img alt="Bulbasaur" src="https://assets.pokemon.com/assets/cms2/img/pokedex/detail/001.png"/>
 <h3 class="match">
         Bulbasaur
         <span class="pokemon-number">
             Nº 0001
         </span>
 </h3>
 <ul class="evolution-attributes match-height">
 <li class="background-color-grass match">Grass</li>
 <li class="background-color-poison match">Poison</li>
 </ul>
 </a>
 </li>
 <li class="middle">
 <a href="/br/pokedex/ivysaur">
 <img alt="Ivysaur" src="https://assets.pokemon.com/assets/cms2/img/pokedex/detail/002.png"/>
 <h3 class="match">
         Ivysaur
         <span class="pokemon-number">
             Nº 0002
         </span>
 </h3>
 <ul class="evolution-attributes match-height">
 <li class="background-color-grass match">Grass</li>
 <li class="background-color-poison match">Poison</li>
 </ul>
 </a>
 </li>
 <li class="last">
 <a href="/br/pokedex/venusaur">
 <i

### Obtendo todo o conteúdo de texto de uma página

In [26]:
#soup.findAll(text=True)

## 4.2. Outros métodos de pesquisa

- ### *findParent(tag, attributes, text, **kwargs)*

- ### *findParents(tag, attributes, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-parents-and-find-parent

> **Observação:**
> - *findParent()* e *findParents()* também podem ser utilizados como *find_parent()* e *find_parents()*, respectivamente.
---
- ### *findNextSibling(tag, attributes, text, **kwargs)*

- ### *findNextSiblings(tag, attributes, text, limit, **kwargs)*

- ### *findPreviousSibling(tag, attributes, text, **kwargs)*

- ### *findPreviousSiblings(tag, attributes, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-next-siblings-and-find-next-sibling
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-previous-siblings-and-find-previous-sibling

> **Observação:**
> - *findNextSibling()*, *findNextSiblings()*, *findPreviousSibling()* e *findPreviousSiblings()* também podem ser utilizados como *find_next_sibling()*, *find_next_siblings()*, *find_previous_sibling()* e *find_previous_siblings()*, respectivamente.
---
- ### *findNext(tag, attributes, text, **kwargs)*

- ### *findAllNext(tag, attributes, text, limit, **kwargs)*

- ### *findPrevious(tag, attributes, text, **kwargs)*

- ### *findAllPrevious(tag, attributes, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all-next-and-find-next
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all-previous-and-find-previous

> **Observação:**
> - *findNext()*, *findAllNext()*, *findPrevious* e *findAllPrevious* também podem ser utilizados como *find_next()*, *find_all_next()*, *find_previous()* e *find_all_previous()*, respectivamente.

### HTML de exemplo para ilustrar a utilização dos métodos de pesquisa do BeautifulSoup

<img src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/BeautifulSoup-method.png" width=80%>

### Example

In [27]:
html_test = """"
<html>
    <body>
        <div class="pokemon_001">
            <h1>Bulbasaur</h1>
            <ul>
                <li>Grass</li>
                <li>Poison</li>
            </ul>l
            <p>Nº 0001</p>
         </div>
        <div class="pokemon_002">
            <h1>Ivysaur</h1>
            <ul>
                <li>Grass</li>
                <li>Poison</li>
            </ul>
            <p>Nº 0002</p>
        </div>
        <div class="pokemon_003">
            <h1>Venusaur</h1>
            <ul>
                <li>Grass</li>
                <li>Poison</li>
            </ul>
            <p>Nº 0003</p>
        </div>
    </body>
</html>
"""

### Read Example HTML

In [28]:
html_test = treat_html(html_test)
html_test

'" <html><body><div class="pokemon_001"><h1>Bulbasaur</h1><ul><li>Grass</li><li>Poison</li></ul>l <p>Nº 0001</p></div><div class="pokemon_002"><h1>Ivysaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0002</p></div><div class="pokemon_003"><h1>Venusaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0003</p></div></body></html>'

In [29]:
soup = BeautifulSoup(html_test, 'html.parser')
soup

" <html><body><div class="pokemon_001"><h1>Bulbasaur</h1><ul><li>Grass</li><li>Poison</li></ul>l <p>Nº 0001</p></div><div class="pokemon_002"><h1>Ivysaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0002</p></div><div class="pokemon_003"><h1>Venusaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0003</p></div></body></html>

### Parents

In [30]:
soup.findAll('h1')

[<h1>Bulbasaur</h1>, <h1>Ivysaur</h1>, <h1>Venusaur</h1>]

In [31]:
soup.find('h1').find_parent('div')

<div class="pokemon_001"><h1>Bulbasaur</h1><ul><li>Grass</li><li>Poison</li></ul>l <p>Nº 0001</p></div>

In [32]:
soup.find('h1').find_parents()

[<div class="pokemon_001"><h1>Bulbasaur</h1><ul><li>Grass</li><li>Poison</li></ul>l <p>Nº 0001</p></div>,
 <body><div class="pokemon_001"><h1>Bulbasaur</h1><ul><li>Grass</li><li>Poison</li></ul>l <p>Nº 0001</p></div><div class="pokemon_002"><h1>Ivysaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0002</p></div><div class="pokemon_003"><h1>Venusaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0003</p></div></body>,
 <html><body><div class="pokemon_001"><h1>Bulbasaur</h1><ul><li>Grass</li><li>Poison</li></ul>l <p>Nº 0001</p></div><div class="pokemon_002"><h1>Ivysaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0002</p></div><div class="pokemon_003"><h1>Venusaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0003</p></div></body></html>,
 " <html><body><div class="pokemon_001"><h1>Bulbasaur</h1><ul><li>Grass</li><li>Poison</li></ul>l <p>Nº 0001</p></div><div class="pokemon_002"><h1>Ivysaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0002</p></div><div class="pokemon_003"><h1

In [33]:
for item in soup.findAll('h1'):
    print(item.find_parent('div'))

<div class="pokemon_001"><h1>Bulbasaur</h1><ul><li>Grass</li><li>Poison</li></ul>l <p>Nº 0001</p></div>
<div class="pokemon_002"><h1>Ivysaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0002</p></div>
<div class="pokemon_003"><h1>Venusaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0003</p></div>


### Siblings

In [34]:
soup.find('h1').findNextSibling()

<ul><li>Grass</li><li>Poison</li></ul>

In [35]:
soup.find('ul').findPreviousSibling()

<h1>Bulbasaur</h1>

In [36]:
soup.find('p').findPreviousSiblings()

[<ul><li>Grass</li><li>Poison</li></ul>, <h1>Bulbasaur</h1>]

### Next e Previous

In [37]:
soup.find('ul').findNext()

<li>Grass</li>

In [38]:
soup.find('ul').findPrevious()

<h1>Bulbasaur</h1>

In [39]:
soup.find('div').findAllNext()

[<h1>Bulbasaur</h1>,
 <ul><li>Grass</li><li>Poison</li></ul>,
 <li>Grass</li>,
 <li>Poison</li>,
 <p>Nº 0001</p>,
 <div class="pokemon_002"><h1>Ivysaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0002</p></div>,
 <h1>Ivysaur</h1>,
 <ul><li>Grass</li><li>Poison</li></ul>,
 <li>Grass</li>,
 <li>Poison</li>,
 <p>Nº 0002</p>,
 <div class="pokemon_003"><h1>Venusaur</h1><ul><li>Grass</li><li>Poison</li></ul><p>Nº 0003</p></div>,
 <h1>Venusaur</h1>,
 <ul><li>Grass</li><li>Poison</li></ul>,
 <li>Grass</li>,
 <li>Poison</li>,
 <p>Nº 0003</p>]

In [40]:
soup.find('li').findNextSiblings('li', limit=1)

[<li>Poison</li>]

# <font color=green>5. POKEDEX WEB SCRAPING

## 5.1. Identificando e selecionando os dados no HTML

### Obtendo o HTML e criando o objeto BeautifulSoup

In [41]:
response = urlopen('https://www.pokemon.com/br/pokedex/')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
#soup

### Criando variávels para armazenar informações

In [55]:
pokemon_cards = []
card = {}

### Criando dicionário com a listagem de pokemons

In [60]:
import re

#Pegando ID e nome dos pokemons, através das tags que possuem href='us/pokedex/'
i = 1
for pokemon in soup.find_all('a', href=re.compile("/us/pokedex/")):
    pokemon_info =  pokemon.get_text()
    pokemon_info = pokemon_info.replace(' ', '')
    pokemon_info = pokemon_info.split('-')

    pokemon_dict = {}
    pokemon_dict['pokemon_id'] = pokemon_info[0]
    pokemon_dict['pokemon_name'] = pokemon_info[1]

    card['pokemon_info_'+str(i)] = pokemon_dict

    i += 1

card

{'pokemon_info_1': {'pokemon_id': '1', 'pokemon_name': 'Bulbasaur'},
 'pokemon_info_2': {'pokemon_id': '2', 'pokemon_name': 'Ivysaur'},
 'pokemon_info_3': {'pokemon_id': '3', 'pokemon_name': 'Venusaur'},
 'pokemon_info_4': {'pokemon_id': '4', 'pokemon_name': 'Charmander'},
 'pokemon_info_5': {'pokemon_id': '5', 'pokemon_name': 'Charmeleon'},
 'pokemon_info_6': {'pokemon_id': '6', 'pokemon_name': 'Charizard'},
 'pokemon_info_7': {'pokemon_id': '7', 'pokemon_name': 'Squirtle'},
 'pokemon_info_8': {'pokemon_id': '8', 'pokemon_name': 'Wartortle'},
 'pokemon_info_9': {'pokemon_id': '9', 'pokemon_name': 'Blastoise'},
 'pokemon_info_10': {'pokemon_id': '10', 'pokemon_name': 'Caterpie'},
 'pokemon_info_11': {'pokemon_id': '11', 'pokemon_name': 'Metapod'},
 'pokemon_info_12': {'pokemon_id': '12', 'pokemon_name': 'Butterfree'},
 'pokemon_info_13': {'pokemon_id': '13', 'pokemon_name': 'Weedle'},
 'pokemon_info_14': {'pokemon_id': '14', 'pokemon_name': 'Kakuna'},
 'pokemon_info_15': {'pokemon_id':

## 5.2. Obtendo o VALOR do veículo anunciado

### <font color=red>Resumo

In [ ]:
# Valor


## 5.3. Obtendo as INFORMAÇÕES sobre o veículo anunciado

### <font color=red>Resumo

In [ ]:
# Informações


## 5.4. Obtendo os ACESSÓRIOS do veículo anunciado

### <font color=red>Resumo

In [ ]:
# Acessórios


## 5.5 Criando um DataFrame com os dados coletados do Alura Motors

## 5.6. Obtendo a FOTO do anúncio

### Visualizando a FOTO no notebook (extra)

### Rotina para acessar e salvar a FOTO do anúncio

### https://docs.python.org/3/library/urllib.request.html#urllib.request.urlretrieve

### <font color=red>Resumo

# <font color=green>6. WEB SCRAPING DO SITE ALURA MOTORS - OBTENDO OS DADOS DE TODOS OS ANÚNCIOS DE UMA PÁGINA

## 6.1. Identificando as informações no HTML

## 6.2. Criando uma rotina de scraping

# <font color=green>7. WEB SCRAPING DO SITE ALURA MOTORS - OBTENDO OS DADOS DE TODOS OS ANÚNCIOS DO SITE

## 7.1. Identificando as informações no HTML

## 7.2. Criando uma rotina de scraping